In [12]:
def dtw(T1, T2):
    
    
    """
    This is code with running time O(|T1|*|T2|) and space O(|T2|)
    It returns only the value for dtw(T1,T2)
    Below is code with running time O(|T1|*|T2|) and space O(|T1|*|T2|), that can also return the monotone assignment
    
    memo = [0] * len(T2)
    memo[0] .append(dist(T1[0], T2[0]))
    for col in range(1, len(T2)):
        memo[i] = memo[col - 1] + dist(T1[0], T2[col])
        
    for row in range(1, len(T1)):
        temp = memo[0]
        memo[0] = memo[0] + dist(T1[row], T2[0])
        for col in range(1, len(T2)):
            store = memo[col]
            
            memo[col] = dist(T1[row], T2[col]) + min(temp, memo[col-1], memo[col])
            temp = store
            
    return memo[-1]
    """
    
    #We store in matrix[row][col][0] the minimal dtw when we assign only the first "row" points in T1 
    #and the first "col" points in T2.
    #matrix[row][col] = dist(T1[row], T2[col]) + min(matrix[row-1][col-1], matrix[row-1][col], matrix[row][col-1])
    
    #Note that an assignment of the first "row" points in T1 and the first "col" points in T2, 
    #always assigns the last point of T1 to the last point of T2.
    #Note as well that the assignment of points is monotone and thus it is nondecreasing in both indices
    #i.e. assignment[i][0] <= assignment[i + 1][0], and assignment[i][1] <= assignment[i + 1][1]
    #That allows us to efficiently store only the differences between assignment[i + 1] - assignment[i]
    
    #Thus we store in matrix[row][col][1] the previous tuple of points in the assignment 
    #E.g. if matrix[row-1][col-1] ==  min(matrix[row-1][col-1], matrix[row-1][col], matrix[row][col-1]),
    #then we asssign (row-1, col-1), and store in matrix[row][col][1] the vector (-1,-1) since (row-1, col-1) = (row, col) + (-1,-1)
    
    #if matrix[row-1][col] ==  min(matrix[row-1][col-1], matrix[row-1][col], matrix[row][col-1]),
    #then we asssign (row-1, col), and store in matrix[row][col][1] the vector (-1,0) since (row-1, col) = (row, col) + (-1,0)
    
    matrix = [[[] for i in range(len(T2))] for j in range(len(T2))]
    matrix[0][0].append(dist(T1[0], T2[0]))
    for col in range(1, len(T2)):
        matrix[0][col].append(matrix[0][col - 1][0] + dist(T1[0], T2[col]))
        matrix[0][col].append((0,-1))
    for row in range(1, len(T1)):
        matrix[row][0].append(matrix[row - 1][0][0] + dist(T1[row], T2[0]))
        matrix[row][0].append((-1,0))
        
    for row in range(1, len(T1)):
        for col in range(1, len(T2)):
            recursive = min(matrix[row-1][col-1], matrix[row-1][col], matrix[row][col-1])
            
            matrix[row][col].append(dist(T1[row], T2[col]) + recursive)
            
            if matrix[row-1][col-1] == recursive:
                matrix[row][col].append((-1,-1))
            elif matrix[row-1][col] == recursive:
                matrix[row][col].append((-1,0))
            else:
                matrix[row][col].append((0,-1))
    
    #The moves that we store in matrix[i][j][1] are essentially pointers to the next tuple of points in @assignment
    #We traverse through those pointers from the bottom right corner (len(T1)- 1, len(T2) -1) until we reach (0,0).
    
    row , col = len(T1) -1, len(T2) -1
    assignment = [(row,col)]
    while (row,col) != (0,0):
        move = matrix[row][col][2]
        row = row + move[0]
        col = col + move[1]
        assignment.append(row,col)
        
    
    return matrix[-1][-1], assignment
        

In [13]:
def dfd(T1, T2):
    
    matrix = [[[] for i in range(len(T2))] for j in range(len(T2))]
    matrix[0][0].append(dist(T1[0], T2[0]))
    for col in range(1, len(T2)):
        matrix[0][col].append(max(matrix[0][col - 1][0], dist(T1[0], T2[col])))
        matrix[0][col].append((0,-1))
    for row in range(1, len(T1)):
        matrix[row][0].append(max(matrix[row -1][col][0], dist(T1[row], T2[0])))
        matrix[row][0].append((-1,0))
        
    for row in range(1, len(T1)):
        for col in range(1, len(T2)):
            recursive = min(matrix[row-1][col-1], matrix[row-1][col], matrix[row][col-1])
            
            matrix[row][col].append(max(dist(T1[row], T2[col]), recursive))
            
            if matrix[row-1][col-1] == recursive:
                matrix[row][col].append((-1,-1))
            elif matrix[row-1][col] == recursive:
                matrix[row][col].append((-1,0))
            else:
                matrix[row][col].append((0,-1))
    
    row , col = len(T1) -1, len(T2) -1
    assignment = [(row,col)]
    while (row,col) != (0,0):
        move = matrix[row][col][2]
        row = row + move[0]
        col = col + move[1]
        assignment.append(row,col)
        
    
    return matrix[-1][-1], assignment

In [14]:
def dist(p,q):
    return (p[0] - q[0]) * (p[0] - q[0]) + (p[1] - q[1]) * (p[1] - q[1])